In [1]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

In [18]:
!pip install beautifulsoup4

In [19]:
!pip install lxml

In [20]:
response=requests.get("https://aceacademyindia.com/bigbluebutton/api/").text

In [23]:
soup=BeautifulSoup(response,'lxml')

In [24]:
print(soup.prettify())

<html>
 <body>
  <response>
   <returncode>
    SUCCESS
   </returncode>
   <version>
    2.0
   </version>
  </response>
 </body>
</html>


In [46]:
import pdb
def get_by_direct_key(soup, key):
    return {f"{key}": soup.find(key).text}

def get_soup(response):
    return BeautifulSoup(response.text, "lxml")
    
def get_response_(soup,keys = ["returncode", "message"]):
    result = dict()
    for i in keys:
        result.update(get_by_direct_key(soup, i))
    return result

def get_meetings_(soup, key1='meeting',key2=['meetingname','meetingid','running']):
    result = list()
    result.append(soup.find('returncode').text)
    for i in soup.find_all(key1):
        result.append(get_response_(i,keys=key2))
    if result[0] != "SUCCESS":
        result[0] = None
    return result



In [4]:
create='https://aceacademyindia.com/bigbluebutton/api/create?allowStartStopRecording=true&attendeePW=ap&autoStartRecording=false&meetingID=random-5410535&moderatorPW=mp&name=random-5410535&record=false&voiceBridge=74568&welcome=%3Cbr%3EWelcome+to+%3Cb%3E%25%25CONFNAME%25%25%3C%2Fb%3E%21&checksum=3f55ccfe443879568e5430e6852406a4612786e9'
meets='https://aceacademyindia.com/bigbluebutton/api/getMeetings?checksum=ee0fbc06ea78e8b00bfd07f30c8a54a3b894bc3d'


In [21]:
res_create=get_response_(get_soup(requests.get(create)))

In [22]:
res_create

{'returncode': 'SUCCESS', 'message': ''}

In [ ]:
rmeets=requests.get(meets)

In [47]:
res_meetings=get_meetings_(get_soup(rmeets))

In [48]:
res_meetings

['SUCCESS',
 {'meetingname': 'random-5410535',
  'meetingid': 'random-5410535',
  'running': 'false'}]

In [24]:

soup=get_soup(rmeets)

In [26]:
a=soup.find_all('meeting')

In [28]:
i=a[0]

In [39]:
i.find('meetingname')

<meetingname>random-5410535</meetingname>

In [40]:
i.find('running')

<running>false</running>

In [30]:
get_response_(i,keys=['meetingname','meetingid','running'])

AttributeError: 'NoneType' object has no attribute 'text'

## Scrapping TickerTape Stock Financials

In [2]:
from requests_html import HTML, HTMLSession
from urllib.parse import urljoin
import pandas as pd

In [92]:
def get_financial_data(url):
    with HTMLSession() as session:
        r = session.get(url)
        financials = r.html.find('.route-tab-header-root a')
        links = { tab.text : urljoin(url,tab.attrs.get('href','')) for tab in financials }
        finance = session.get(links['Financials'])
        cols = finance.html.find('.financials-table-container .col')
        data = {}
        for i,col in enumerate(cols):
            values = col.text.split('\n')
            if len(values)>1:
                data[i] = values
        data_ = pd.DataFrame(data)
        data_.columns = data_.iloc[0,:]
        data_ = data_.drop(0,axis=0)
    return data_

In [93]:
base_url = "https://www.tickertape.in/stocks/equitas-holdings-EQHL"
data_eqhl =  get_financial_data(base_url)

In [94]:
data_eqhl.head()

,Financial Year,FY 2012,FY 2013,FY 2014,FY 2015,FY 2016,FY 2017,FY 2018,FY 2019,FY 2020,FY 2021
1,Total Revenue,198.65,283.17,483.52,755.92,"1,114.88","1,556.55","1,779.68","2,358.53","2,923.36","3,820.69"
2,Raw Materials,—,—,—,—,—,—,—,—,—,—
3,Power & Fuel Cost,—,—,—,—,—,—,—,—,—,—
4,Employee Cost,63.83,78.05,100.31,155.10,233.80,396.09,549.85,567.65,725.57,813.86
5,Selling & Administrative Expenses,44.88,48.19,72.27,78.18,102.60,163.01,287.13,289.64,267.62,347.14


In [95]:
data_hdfc = get_financial_data("https://www.tickertape.in/stocks/hdfc-bank-HDBK")

In [96]:
data_hdfc

,Financial Year,FY 2012,FY 2013,FY 2014,FY 2015,FY 2016,FY 2017,FY 2018,FY 2019,FY 2020,FY 2021
1,Total Revenue,"34,185.72","42,993.98","50,852.52","60,212.17","74,373.21","86,176.43","1,01,344.44","1,24,107.79","1,47,068.26","1,55,885.27"
2,Raw Materials,—,—,—,—,—,—,—,—,—,—
3,Power & Fuel Cost,—,—,—,—,—,—,—,—,—,—
4,Employee Cost,"3,573.09","4,201.79","4,494.47","5,162.68","6,306.14","8,504.70","9,193.90","10,451.15","12,920.13","13,676.67"
5,Selling & Administrative Expenses,"2,391.44","2,891.77","3,019.28","3,463.77","4,087.83","4,603.71","4,976.65","5,187.30","5,489.56","6,435.33"
6,Operating & Other expenses,"19,999.54","25,233.15","29,439.44","34,825.84","43,730.23","48,788.87","57,743.18","72,930.51","89,186.95","91,592.12"
7,EBITDA,"8,221.65","10,667.27","13,899.33","16,759.88","20,249.01","24,279.15","29,430.71","35,538.83","39,471.62","44,181.15"
8,Depreciation/Amortization,554.16,663.26,688.68,680.45,738.03,886.19,966.78,"1,220.67","1,276.77","1,385.01"
9,PBIT,"7,667.49","10,004.01","13,210.65","16,079.43","19,510.98","23,392.96","28,463.93","34,318.16","38,194.85","42,796.14"
10,Interest & Other Items,—,—,—,—,—,—,—,—,—,—


In [97]:
data_icici = get_financial_data("https://www.tickertape.in/stocks/icici-bank-ICBK")

In [98]:
data_icici

,Financial Year,FY 2012,FY 2013,FY 2014,FY 2015,FY 2016,FY 2017,FY 2018,FY 2019,FY 2020,FY 2021
1,Total Revenue,"66,658.28","74,204.40","79,563.86","90,216.24","1,01,395.85","1,13,397.63","1,18,969.10","1,31,306.50","1,49,786.10","1,61,336.47"
2,Raw Materials,—,—,—,—,—,—,—,—,—,—
3,Power & Fuel Cost,—,—,—,—,—,—,—,—,—,—
4,Employee Cost,"5,101.27","5,629.09","5,968.79","6,568.32","6,912.29","7,893.26","8,333.53","9,425.26","11,156.75","11,050.91"
5,Selling & Administrative Expenses,"2,784.22","7,469.63","7,986.11","3,693.91","4,168.82","4,896.26","5,704.35","7,210.07","8,164.61","8,593.95"
6,Operating & Other expenses,"47,420.86","46,870.55","48,608.17","60,821.86","75,167.22","85,887.12","93,030.62","1,06,317.07","1,10,704.92","1,14,323.21"
7,EBITDA,"11,351.93","14,235.13","17,000.79","19,132.15","15,147.52","14,720.99","11,900.60","8,354.10","19,759.82","27,368.40"
8,Depreciation/Amortization,671.44,625.48,719.27,798.22,843.11,911.64,922.14,945.84,"1,171.22","1,340.07"
9,PBIT,"10,680.49","13,609.65","16,281.52","18,333.93","14,304.41","13,809.35","10,978.46","7,408.26","18,588.60","26,028.33"
10,Interest & Other Items,—,—,—,—,—,—,—,—,—,—
